The particle dynamics in a general velocity field $ \mathbf{u}(\mathbf{x}, t) = \begin{pmatrix} u((\mathbf{x}, t)) \\ v((\mathbf{x}, t)) \end{pmatrix} $. This function evaluates the velocity field $ \mathbf{u}(\mathbf{x}, t) $, at point $ \mathbf{x} $ at time $ t $.

| Name | Type (Shape) | Description |
| --- | --- | --- |
| t | float | time |
| x | array (3,) | $ \mathbf{x} $ |
| X | array (Nx, Ny) | X-meshgrid|
| Y | array (Nx, Ny) | Y-meshgrid|
| Interpolant | list (3,) | Interpolant[0]: Interpolant object for $ u((\mathbf{x}, t))  $ <br /> Interpolant[1]: Interpolant object for $ v((\mathbf{x}, t))  $. <br /> <br /> The interpolation, however, works only <br /> if the particle is in the defined flow domain. <br /> At the boundary of the domain, <br /> the interpolant is not defined.|
| periodic | list (3,) | periodic[0]: periodicity in x <br /> periodic[1]: periodicity in y <br />|
| defined_domain | array (Nx, Ny) | points on the meshgrid where velocity field is defined |
| bool_unsteady | bool | specifies if velocity field is unsteady/steady |
| dt_data | float | time spacing of the velocity data |
| delta | list (3,) | delta[0]: x-grid spacing of the velocity data <br /> delta[1]: y-grid spacing of the velocity data|
| u | float | $ u((\mathbf{x}, t)) $ |
| v | float | $ v((\mathbf{x}, t)) $ |
| dDF11dt | float | $ \dfrac{dDF_{11}}{dt} $ |
| dDF12dt | float | $ \dfrac{dDF_{12}}{dt} $ |
| dDF21dt | float | $ \dfrac{dDF_{21}}{dt} $ |
| dDF22dt | float | $ \dfrac{dDF_{22}}{dt} $ |

In [1]:
import sys, os

# get current directory
path = os.getcwd()

# get parent directory
parent_directory = os.path.sep.join(path.split(os.path.sep)[:-1])

# add Algorithm folder to current working path in order to access the functions
sys.path.append(parent_directory+"/utils")

In [2]:
# Import numpy
import numpy as np

# Import function that computes velocity
from ipynb.fs.defs.velocity import velocity

# Import function that computes gradient of velocity
from ipynb.fs.defs.gradient_velocity import gradient_velocity

In [3]:
def dxdt_eqVar(t, y, X, Y, Interpolant, periodic, defined_domain, bool_unsteady, dt_data, delta, aux_grid):
    
    # x/y position of particle
    x = y[:2]
    
    # first row of DF
    DF11, DF12, DF13 = y[2], y[3], y[4]
    
    # second row of DF
    DF21, DF22, DF23 = y[5], y[6], y[7]
    
    # second row of DF
    DF31, DF32, DF33 = y[8], y[9], y[10]
    
    DF = np.array([[y[2], y[3], y[4]], [y[5], y[6], y[7]], [y[8], y[9], y[10]]])
    
    dx_data = delta[0]
    dy_data = delta[1]
    dz_data = delta[2]
    
    # check if periodic in x
    if periodic[0]:
        
        x[0] = x[0]%(X[0, -1, 0]-X[0, 0, 0])
    
    # check if periodic in y
    if periodic[1]:
        
        x[1] = x[1]%(Y[-1, 0, 0]-Y[0, 0, 0])
        
    # check if periodic in y
    if periodic[2]:
        
        x[2] = x[2]%(Z[0, 0, -1]-Z[0, 0, 0])
    
    vel = velocity(t, x, X, Y, Z, Interpolant, periodic, defined_domain, bool_unsteady, dt_data, delta)
    
    grad_vel = gradient_velocity(t, x, X, Y, Z, Interpolant, periodic, defined_domain, bool_unsteady, dt_data, delta, aux_grid)
    
    dDFdt = grad_vel@DF
    
    u = vel[0]
    v = vel[1]
    w = vel[2]
    dDF11dt, dDF12dt = dDFdt[0,0], dDFdt[0, 1], dDFdt
    dDF12dt = dDFdt[0,1]
    dDF21dt = dDFdt[1,0]
    dDF22dt = dDFdt[1,1]
    
    return [u, v, w, dDF11dt, dDF12dt, dDF21dt, dDF22dt]